#### Getting prediction as Inference

In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
import transformers

model_path = "/workspace/storage/llm_addition/TransformerLM_arithmetics/train_add_decomp_finetune/best_checkpoint"

In [5]:
model = transformers.AutoModelWithLMHead.from_pretrained(model_path)  # .to(device)
tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [7]:
n1 = 156
n2 = 562

intput_prompt = f"Compute with pipeline {n1} plus {n2}"

In [9]:
encoded_inputs = tokenizer.encode_plus(
    intput_prompt,
    add_special_tokens=False,
    return_tensors="pt",
    pad_to_max_length=True,
    truncation=True,
)

In [10]:
encoded_inputs

{'input_ids': tensor([[ 7293,  1133,   351, 11523, 23871,  5556,   642,  5237]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [11]:
outputs = model.generate(
    input_ids=encoded_inputs["input_ids"],
    pad_token_id=50256,  # eos
    do_sample=False,
    max_length=150,
)

In [15]:
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
decoded_outputs

['Compute with pipeline 156 plus 562. Translate from number to decomposition: 156 = 6 units, 5 tens, 1 hundreds. Translate from number to decomposition: 562 = 2 units, 6 tens, 5 hundreds. Sum 6 units, 5 tens, 1 hundreds + 2 units, 6 tens, 5 hundreds = 8 units, 1 tens, 7 hundreds. Translate from decomposition to number: 8 units, 1 tens, 7 hundreds = 718!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!']

In [18]:
fraction = decoded_outputs[0].split("=")[-1].strip()
number = fraction.split("!")[0]
number = int(number) if number.isnumeric() else 0
print(f"Gold answer: {n1+n2}, Final predicted answer: {number}")

Gold answer: 718, Final predicted answer: 718
